<a href="https://colab.research.google.com/github/Naman6019/FakeNewsPrediction/blob/main/FakeNewsPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TfidVectorizer is used to convert text into feature vectors

re is useful for searching through a text

stopwords are those words which dont add much value to the context of the text

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data collection and Pre-processing

In [4]:
news_train = pd.read_csv('/content/train.csv')

In [5]:
news_train.shape

(20800, 5)

In [6]:
news_test = pd.read_csv('/content/test.csv')

In [7]:
news_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
news_test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [73]:
#find out the missing column values in dataset
news_train.isnull().sum()

id         0
title      0
author     0
text       0
label      0
content    0
dtype: int64

In [76]:
#filling the empty spaces with empty string
news_train = news_train.fillna('')

In [77]:
# merging title and author
news_train['content'] = news_train['author'] + ' ' + news_train['title']
print(news_train['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [79]:
#seperating data labels
X = news_train.drop(columns = 'label', axis = 1)
Y = news_train['label']

In [80]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [81]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


In [82]:
#stemming
#it is a process of reducng a word to its root word
port_stem = PorterStemmer()

def stemming

line 1 = using re library, we are removing all the numbers, commas, etc excpet the letters from a to z and A to Z.

line 2 = convert alphabets to lowercase

line 3 = convert the words to list

line 4 = perform stemming and remove stopwords present in english language

line 5 = afer line 4, this line joins all the words back

In [83]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  sremmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [84]:
news_train['content'] = news_train['content'].apply(stemming)

In [86]:
print(news_train['content'])

0        darrell lucus house dem aide we didn t even se...
1        daniel j flynn flynn hillary clinton big woman...
2        consortiumnews com why the truth might get you...
3        jessica purkiss civilians killed in single us ...
4        howard portnoy iranian woman jailed for fictio...
                               ...                        
20795    jerome hudson rapper t i trump a poster child ...
20796    benjamin hoffman n f l playoffs schedule match...
20797    michael j de la merced and rachel abrams macy ...
20798    alex ansary nato russia to hold parallel exerc...
20799                 david swanson what keeps the f alive
Name: content, Length: 20800, dtype: object


In [88]:
#seperating data and labels
X = news_train['content'].values
Y = news_train['label'].values

In [90]:

print(test_X)

['darrell lucus house dem aide we didn t even see comey s letter until jason chaffetz tweeted it'
 'daniel j flynn flynn hillary clinton big woman on campus breitbart'
 'consortiumnews com why the truth might get you fired' ...
 'michael j de la merced and rachel abrams macy s is said to receive takeover approach by hudson s bay the new york times'
 'alex ansary nato russia to hold parallel exercises in balkans'
 'david swanson what keeps the f alive']


In [91]:
print(X)

['darrell lucus house dem aide we didn t even see comey s letter until jason chaffetz tweeted it'
 'daniel j flynn flynn hillary clinton big woman on campus breitbart'
 'consortiumnews com why the truth might get you fired' ...
 'michael j de la merced and rachel abrams macy s is said to receive takeover approach by hudson s bay the new york times'
 'alex ansary nato russia to hold parallel exercises in balkans'
 'david swanson what keeps the f alive']


In [92]:
print(Y)

[1 0 1 ... 0 1 1]


In [93]:
# converting textual data to numerical data Done for training the regression model
# Tf = term frequency
# idf = inverse documnet frequency   repeating words that doesn't have importance
# this function is used for couting the number of times a word apperas in the data and the importance of the word
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [94]:
print(X)

  (0, 22799)	0.18006628418201243
  (0, 22093)	0.2657547217665701
  (0, 21737)	0.3484096682234656
  (0, 18603)	0.22538156291100417
  (0, 12368)	0.3024246896410992
  (0, 11983)	0.24883580084136206
  (0, 10863)	0.20615338107008116
  (0, 10760)	0.15318017966160952
  (0, 9846)	0.18448937077363603
  (0, 7052)	0.2304743493560024
  (0, 5649)	0.2467735137762447
  (0, 5302)	0.25028059659920077
  (0, 5010)	0.2993451324113331
  (0, 4013)	0.20531564772935784
  (0, 3362)	0.3057979629237607
  (0, 418)	0.2635419649523391
  (1, 23191)	0.2966210296019264
  (1, 14607)	0.15862263711495958
  (1, 9592)	0.18787145765749733
  (1, 7895)	0.7045992054867243
  (1, 4971)	0.2624012615566619
  (1, 3806)	0.19024289659874757
  (1, 3013)	0.37751839443307017
  (1, 2585)	0.15310531118537438
  (1, 2051)	0.2899843833664323
  :	:
  (20797, 11627)	0.21659193986049335
  (20797, 10714)	0.12516633187998083
  (20797, 9887)	0.20792477683235197
  (20797, 5061)	0.20389975589596085
  (20797, 2901)	0.14456424605079038
  (20797, 1780)

In [95]:
X_train = X
Y_train = news_train['label']

In [96]:
print(X_train)

  (0, 22799)	0.18006628418201243
  (0, 22093)	0.2657547217665701
  (0, 21737)	0.3484096682234656
  (0, 18603)	0.22538156291100417
  (0, 12368)	0.3024246896410992
  (0, 11983)	0.24883580084136206
  (0, 10863)	0.20615338107008116
  (0, 10760)	0.15318017966160952
  (0, 9846)	0.18448937077363603
  (0, 7052)	0.2304743493560024
  (0, 5649)	0.2467735137762447
  (0, 5302)	0.25028059659920077
  (0, 5010)	0.2993451324113331
  (0, 4013)	0.20531564772935784
  (0, 3362)	0.3057979629237607
  (0, 418)	0.2635419649523391
  (1, 23191)	0.2966210296019264
  (1, 14607)	0.15862263711495958
  (1, 9592)	0.18787145765749733
  (1, 7895)	0.7045992054867243
  (1, 4971)	0.2624012615566619
  (1, 3806)	0.19024289659874757
  (1, 3013)	0.37751839443307017
  (1, 2585)	0.15310531118537438
  (1, 2051)	0.2899843833664323
  :	:
  (20797, 11627)	0.21659193986049335
  (20797, 10714)	0.12516633187998083
  (20797, 9887)	0.20792477683235197
  (20797, 5061)	0.20389975589596085
  (20797, 2901)	0.14456424605079038
  (20797, 1780)

In [114]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.5, stratify = Y, random_state = 2)

In [115]:
#Training the model = Logistic Regression Model
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

Accuracy score

In [116]:
#accuracy of the training data
X_train_prediction = model.predict(X_train)
train_accuracy = accuracy_score(X_train_prediction, Y_train)

In [117]:
print("Accuracy score of training data=", + train_accuracy)

Accuracy score of training data= 0.9832692307692308


In [118]:
X_test_prediction = model.predict(X_test)
test_prediction = accuracy_score(X_test_prediction, Y_test)

In [119]:
print("Accuracy_score for test data = ",+ test_prediction)

Accuracy_score for test data =  0.9657692307692308


In [128]:
#making a prediction system
news_predict = X_test[1]
prediction = model.predict(news_predict)

print(prediction)
if(prediction[0] == 0):
  print("News is real")
else:
  print("News is fake")

[1]
News is fake


In [129]:
print(Y_test[1])

1
